# Module 2.- EDA

In [12]:
import pandas as pd
import os
import pyarrow
import contextlib
import io
from collections import Counter

## Load BDD

In [2]:
PATH = '../../data/'
parquet_files = ["abandoned_carts.parquet","inventory.parquet","orders.parquet", "regulars.parquet", "users.parquet"]

def get_info_middle(df):
    fobj = io.StringIO()
    with contextlib.redirect_stdout(fobj) as stdout:
        df.info()
    fobj.seek(0)
    return '\n'.join(fobj.read().splitlines()[3:-2])

abandoned_carts = pd.read_parquet(PATH + parquet_files[0], engine='pyarrow')
inventory = pd.read_parquet(PATH + parquet_files[1], engine='pyarrow')
orders = pd.read_parquet(PATH + parquet_files[2], engine='pyarrow')
regulars = pd.read_parquet(PATH + parquet_files[3], engine='pyarrow')
users = pd.read_parquet(PATH + parquet_files[4], engine='pyarrow')

In [ ]:
print('#####Abandoned######\n#####')
print(abandoned_carts.info())
print('#####Shape######')
print(abandoned_carts.shape)

print('#####Inventory#####\n#####')
print(inventory.info())
print('#####Shape######')
print(inventory.shape)

In [ ]:

print('#####Orders######\n#####')
print(orders.info())
print('#####Shape######')
print(orders.shape)

print('#####Regulars#####\n#####')
print(regulars.info())
print('#####Shape######')
print(regulars.shape)

In [ ]:
print('#####Users#####\n#####')
print(users.info())
print('#####Shape######')
print(users.shape)

Los datatypes de cada variable son correctos. La unica tabla que contiene Nulls es la de Users, las variables que describen al grupo de usuarios de compra (People, Adults, Children, Babies, Pets), donde People es la que engloba a las demás (sin contar Pets). Para las tablas de ordenes/abandonados los productos se identifican entre comas ','. Vamos a comprobar que no haya duplicidades en registros que deberian ser unicos. 

In [ ]:
def unique_val(dataframe, columna):
    return dataframe[columna].nunique() == dataframe.shape[0]
print('#####Abandoned######')
print(unique_val(abandoned_carts, 'id'))
print('#####Inventoy######')
print(unique_val(inventory, 'variant_id'))
print('#####Orders######')
print(unique_val(orders, 'id'))
print('#####Regulars######')
print(unique_val(regulars, 'user_id'))
print('#####Users######')
print(unique_val(users, 'user_id'))

Con esto Podemos ver que un usuario puede pedir mas de un item como consumo habitual. Comprobaremos tmabien que todos los items del inventario estan en alguna de las tablas que conteng items (es decir, no hay itmes que no estan registrados).

In [ ]:
def check_items(orders_df, regulars_df, abandoned_carts_df, inventory_df):
    items_inventory = inventory_df['variant_id'].unique()

    orders_df = orders_df.explode('ordered_items')
    abandoned_carts_df = abandoned_carts_df.explode('variant_id')
    items_orders = orders_df['ordered_items'].unique()
    items_regulars = regulars_df['variant_id'].unique()
    items_abandoned_carts = abandoned_carts_df['variant_id'].unique()
    problemas = {}
    if len(items_orders) > len(items_inventory):
        problemas['orders'] = [len(items_orders), len(items_inventory)]
    else:
        problemas['orders'] = 'correcto'    
    if len(items_regulars) > len(items_inventory):
        problemas['regulars'] = [len(items_regulars), len(items_inventory)]
    else:
        problemas['regulars'] = 'correcto'
    if len(items_abandoned_carts) > len(items_inventory):
        problemas['abandoned_carts'] = [len(items_abandoned_carts), len(items_inventory)]
    else:
        problemas['abandoned_carts'] = 'correcto'
    return problemas

resultados = check_items(orders_df= orders, regulars_df=regulars, abandoned_carts_df=abandoned_carts, inventory_df=inventory)
print(resultados)

Para las tablas orders y regulars encontramos items que no estan ergistrados en el inventario y será dificil trackear sus nombres, tipo y propiedades. Esto puede ser debido a un problema al identificar los productos al pasarlos por el lector o que sean items que se externalizan a proveedores no registrados. Tambien que un mismo item esté identificado dos veces. Se tendra en cuenta en prompts siguentes para no dar informacion de items que no se registren. Idealmente haremos lo mismo para los usuarios.

In [ ]:
def check_users(users_df, orders_df, regulars_df, abandoned_carts_df):
    check_users = users_df['user_id'].unique()
    check_orders = orders_df['user_id'].unique()
    check_regulars = regulars_df['user_id'].unique()
    check_abandoned_carts = abandoned_carts_df['user_id'].unique()
    problemas = {}
    if len(check_orders) > len(check_users):
        problemas['orders'] = [len(check_orders), len(check_users)]
    else:
        problemas['orders'] = 'correcto'    
    if len(check_regulars) > len(check_users):
        problemas['regulars'] = [len(check_regulars), len(check_users)]
    else:
        problemas['regulars'] = 'correcto'
    if len(check_abandoned_carts) > len(check_users):
        problemas['abandoned_carts'] = [len(check_abandoned_carts), len(check_users)]
    else:
        problemas['abandoned_carts'] = 'correcto'
    return problemas

resultados = check_users(users_df=users, orders_df=orders, regulars_df=regulars, abandoned_carts_df=abandoned_carts)
print(resultados)

Los usuarios estan, por contra, todos identificados.

## EDA

A continuación se presentan algunas hipótesis o análisis descriptivo de los datos que pretenden responder a ciertas preguntas para entender mejor el negocio.

### Items más vendidos

En esta seccion obtendremos un head de los items mas vendidos y las clase de item más vendido (en toda la historia de la tienda). 

In [ ]:
def most_item(orders_df, inventory_df):
    orders_df = orders_df.explode('ordered_items')
    item_counts = orders_df['ordered_items'].value_counts().reset_index()
    item_counts.columns = ['Item_ID', 'Cantidad']
    merged_df = item_counts.merge(inventory_df, left_on='Item_ID', right_on='variant_id', how='left')
    return merged_df[merged_df['variant_id'].notnull()]
result1 = most_item(orders_df=orders, inventory_df=inventory)
print('Los items mas vendidos en la histira de la tienda son:')
print(result1.head())

In [ ]:
print(result1.tail())

Interesante ver como de los productos mas vendidos los rollos de papel son de dos proveedores diferentes (plenty y whogivesacrap), siendo el segundo un 35% más caro; vale la pena hacer un analiss de sensibilidad en precios de ese item. Al listar los items menos vendidos encontramos items de afeitado. Podemos cuestionar seguir vendiendolos (teniendo encuenta que es solo un exploratory data-analysis)

### Cesta promedio
Par plantear la cesta promedio podemos hacer simplemente un group de la lista de items de cada order pero hemos de tener en cuenta que es facil que se nos escapen cestas que consideremos parecidas al no contener exactamente los mismos items

In [35]:
def common_cart(orders_df, inventory_df):
    orders_df['cesta'] = orders_df['ordered_items'].astype(str)

    # Realizar un groupby en la columna 'cesta' para contar cuántas veces se repite cada cesta
    cestas_contadas = orders_df.groupby('cesta').size().reset_index(name='cantidad')
    cesta_mas_comun = cestas_contadas.loc[cestas_contadas['cantidad'].idxmax()]['cesta']
    # Filtrar el DataFrame original para obtener solo la fila de la cesta más común
    cesta_df = orders_df[orders_df['cesta'] == cesta_mas_comun].head(1)
    cesta_explotada = cesta_df.explode('ordered_items')
    #cONTAMOS cantidad
    cesta_con_cantidad = cesta_explotada.groupby('ordered_items').size().reset_index(name='cantidad')
    cesta_con_precios = cesta_con_cantidad.merge(inventory_df, left_on='ordered_items', right_on='variant_id', how='left')
    cesta_con_precios['precio_total'] = cesta_con_precios['cantidad'] * cesta_con_precios['price']

    #Obtener el nombre de los ítems de la cesta promedio y el precio total de la cesta
    nombres_cesta = cesta_con_precios['tags'].tolist()
    precio_total_cesta = cesta_con_precios['precio_total'].sum()

    return nombres_cesta,precio_total_cesta
nombres_cesta = common_cart(orders_df=orders, inventory_df=inventory)
print("Cesta Promedio:", nombres_cesta)
print("Precio Total de la Cesta Promedio:", precio_total_cesta)


Cesta Promedio: ([array(['oat-milk', 'vegan'], dtype=object)], 43.16)
Precio Total de la Cesta Promedio: 43.16


Parece que la cesta mas comun es comprar cuatro bricks de leche